### This genetic algorithm is an adaptation of [MourvanZhou's evolutionary algorithm code](https://github.com/MorvanZhou/Evolutionary-Algorithm/blob/master/tutorial-contents/Genetic%20Algorithm/Genetic%20Algorithm%20Basic.py) ###

In [3]:
from qutip import *
from scipy import arcsin, sqrt, pi
import numpy as np
import scipy.sparse as sp
from qutip.qobj import Qobj
%matplotlib
import matplotlib.pyplot as plt
import itertools
import copy
from math import log2, log
from scipy.optimize import minimize
import time
from IPython.core.display import clear_output

Using matplotlib backend: Qt5Agg


In [46]:
INPUT_STATES = 4 # at most 16

In [47]:
def h(x):  #binary entropy
    return -x*log2(x) - (1-x)*log2(1-x)

def calc_XN(p):
    def calc_XN_tmp(q):
        tmp = h(q * (1-p)) - h(0.5*(1+sqrt(1-4*p*(1-p)*q**2)))
        return 1/tmp

    res = minimize(calc_XN_tmp, np.array([0.5]), method='nelder-mead', 
                       options={'xtol': 1e-8, 'disp': False});
    return 1/res.fun;


def rand_herm_mod(N, X, Y, density=1, dims=None):      #this is a modified version of the rand_herm function from QuTip
#    if dims:
#        _check_dims(dims, N, N)
    # to get appropriate density of output
    # Hermitian operator must convert via:
    herm_density = 2.0 * arcsin(density) / pi

    X_int = sp.rand(N, N, herm_density, format='csr')
    X_int.data = X - 0.5
    Y_int = X_int.copy()
    Y_int.data = 1.0j * Y - (0.5 + 0.5j)
    X_int = X_int + Y_int
    X_int.sort_indices()
    X_int = Qobj(X_int)
    if dims:
        return Qobj((X_int + X_int.dag()) / 2.0, dims=dims, shape=[N, N])
    else:
        return Qobj((X_int + X_int.dag()) / 2.0)
    

def rand_unitary_mod(N, X, Y, density=1, dims=None):    #this is a modified version of the rand_unitary function from QuTip
    #if dims:
    #    _check_dims(dims, N, N)
    U = (-1.0j * rand_herm_mod(N, X, Y, density)).expm()
    U.data.sort_indices()
    if dims:
        return Qobj(U, dims=dims, shape=[N, N])
    else:
        return Qobj(U)
    


def rand_ket(N, X, Y, density=1, dims=None):

#    if dims:
#        _check_dims(dims, N, 1)
    Xtmp = sp.rand(N, 1, density, format='csr')
    Xtmp.data = X - 0.5
    Ytmp = Xtmp.copy()
    Ytmp.data = 1.0j * Y - (0.5 + 0.5j)
    Xtmp = Xtmp + Ytmp
    Xtmp.sort_indices()
    Xtmp = Qobj(Xtmp)
    if dims:
        return Qobj(Xtmp / Xtmp.norm(), dims=dims, shape=[N, 1])
    else:
        return Qobj(Xtmp / Xtmp.norm())


def A_p2(rho): # this is the amplitude dampening channel on 2 qubits (2 uses). rho is a 2-qubit state.
    #------Moved to main to optimize code
    #KA = tensor(K1,K1)
    #KA_dag = tensor(K1,K1).dag()
    #KB = tensor(K1,K2)
    #KB_dag = tensor(K1,K2).dag()
    #KC = tensor(K2,K1)
    #KC_dag = tensor(K2,K1).dag()
    #KD = tensor(K2,K2)
    #KD_dag = tensor(K2,K2).dag()
    #--------

    return  KA * rho * KA_dag + KB * rho * KB_dag + KC * rho * KC_dag + KD * rho * KD_dag 


def EC(rho):   #qubit erasure channel  --  1 use
    return (1-p) * tensor(rho, basis(2,0)) + p * tensor(rho, basis(2,1))


def depo(rho):     #depolarizing channel -- 2 uses
    left_use = (1-0.75*p)* rho + p/4 * (tensor(sigmax(), identity(2)) * rho * tensor(sigmax(), identity(2)) + tensor(sigmay(), identity(2)) * rho * tensor(sigmay(), identity(2)) + tensor(sigmaz(), identity(2)) * rho * tensor(sigmaz(), identity(2)))
    right_use = (1-0.75*p) * left_use + p/4 * (tensor(identity(2), sigmax()) * left_use * tensor(identity(2),sigmax()) + tensor(identity(2), sigmay()) * left_use * tensor(identity(2), sigmay()) + tensor(identity(2), sigmaz()) * left_use * tensor(identity(2), sigmaz()))
    return right_use

def depo_2(rho):  #another implementation
    first_term = p**2 * rho
    second_term = p*(1-p) * tensor(identity(2)/2, rho.ptrace([1]))
    third_term = p * (1-p) * tensor(rho.ptrace([0]), identity(2)/2)
    fourth_term = (1-p)**2 * tensor(identity(2)/2, identity(2)/2)
    
    return first_term + second_term + third_term + fourth_term
    


def Func(kets,ps):   #This is the estimate of X(N \tensor N)
    #rho_init = tensor(basis(2,0), basis(2,0)) * tensor(basis(2,0), basis(2,0)).dag()   #moved to main
    
    XNN = np.array([( entropy_vn(sum(px * A_p2(ket2dm(ketx)) for px,ketx in zip(ps[i],kets[i])), base = 2) 
                    - sum(px * entropy_vn(A_p2(ket2dm(ketx)), base = 2) for px,ketx in zip(ps[i],kets[i])) ) for i in range(len(ps))])
    
    #print(entropy_vn(sum(px * depo_2(ket2dm(ketx)) for px,ketx in zip(ps[1],kets[1]))) )

    return XNN
    
# find non-zero fitness for selection.
def get_fitness(pred): return pred - np.min(pred)


def translateDNA(pop):   #pop is a list of 2 np arrays. One nparray for the population of X and Y, and one nparray for the population of ps
    XY = pop[0].dot(2 ** np.arange(DNA_SIZE)[::-1]) / (2**DNA_SIZE-1)
    #Us = np.array([[rand_unitary_mod(4, XY[i,j,0], XY[i,j,1], density=1, dims=[[2,2], [2,2]])  for j in range(INPUT_STATES)]  for i in range(POP_SIZE)]) 
    kets = np.array([[rand_ket(4, XY[i,j,0], XY[i,j,1], dims=[[2,2], [1,1]] )  for j in range(INPUT_STATES)]  for i in range(pop[0].shape[0])])
    prob = pop[1].dot(2 ** np.arange(DNA_SIZE)[::-1]) / (2**DNA_SIZE-1)
    #for i in range(POP_SIZE):
    #    for j in range(INPUT_STATES):
    #        prob[i][j] = 0.25
    ps = np.array([prob[i] / sum(prob[i]) for i in range(pop[0].shape[0])] )   
    #for i in range(POP_SIZE):
    #    kets[i,0] = tensor(basis(2,0),basis(2,0))   
    #    kets[i,1] = tensor(basis(2,0),basis(2,1))
    #    kets[i,2] = tensor(basis(2,1),basis(2,0))
    #    kets[i,3] = tensor(basis(2,1),basis(2,1))
    #print("test ", pop[0].shape)
    return [kets,np.sort(ps)]  #Us is an array of POP_SIZE arrays of 16 unitaries each

# nature selection wrt pop's fitness.
def select(pop, fitness):    
    idx = np.random.choice(np.arange(pop[0].shape[0]), size=POP_SIZE, replace=True, p=fitness/fitness.sum())
    return [pop[0][idx], pop[1][idx]]

# mating process (genes crossover).
def crossover_and_mutate(individual_XY, individual_p, pop, mutate=False):
    #crossover
    if np.random.rand() < CROSS_RATE:
        i_ = np.random.randint(0, POP_SIZE, size=1)[0]                        # select another individual from pop
        cross_points_XY = np.random.randint(0, 2, size=INPUT_STATES*2*4*DNA_SIZE).astype(bool).reshape((INPUT_STATES,2,4,DNA_SIZE))# choose crossover points
        cross_points_p = np.random.randint(0, 2, size=DNA_SIZE*INPUT_STATES).astype(bool).reshape((INPUT_STATES,DNA_SIZE))# choose crossover points
        individual_XY[cross_points_XY] = pop[0][i_][cross_points_XY]
        individual_p[cross_points_p] = pop[1][i_][cross_points_p]
        
    #mutate
    if mutate:
        x_XY = np.random.choice([0, 1], size=INPUT_STATES*2*4*DNA_SIZE, 
                             p=[1-MUTATION_RATE, MUTATION_RATE]).astype(bool).reshape((INPUT_STATES,2,4,DNA_SIZE))
                                                      
        x_p = np.random.choice([0, 1], size=INPUT_STATES*DNA_SIZE, 
                             p=[1-MUTATION_RATE, MUTATION_RATE]).astype(bool).reshape((INPUT_STATES,DNA_SIZE))
                                                      

        individual_XY[x_XY] = np.abs(individual_XY[x_XY] - 1)  #flip the bits
        individual_p[x_p] = np.abs(individual_p[x_p] - 1)  #flip the bits
        
    return [individual_XY, individual_p]


In [53]:
p = 0.6                #probability p in the amp. damp. channel
DNA_SIZE = 10         # DNA length   # size of each number in the lists X and Y
POP_SIZE = 10          # population size
CROSS_RATE = 0.001        # mating probability (DNA crossover)
MUTATION_RATE = 0.009    # mutation probability
MUTATION_DISTANCE = 1
N_GENERATIONS = 10000

#---------------Moved from functions for optimization
K1 = basis(2,0) * basis(2,0).dag() + sqrt(1-p) * basis(2,1) * basis(2,1).dag()
K2 = sqrt(p) * basis(2,0) * basis(2,1).dag()
KA = tensor(K1,K1)
KA_dag = tensor(K1,K1).dag()
KB = tensor(K1,K2)
KB_dag = tensor(K1,K2).dag()
KC = tensor(K2,K1)
KC_dag = tensor(K2,K1).dag()
KD = tensor(K2,K2)
KD_dag = tensor(K2,K2).dag()

#rho_init = tensor(basis(2,0), basis(2,0)) * tensor(basis(2,0), basis(2,0)).dag()

bs = (tensor(basis(2), basis(2))+tensor(basis(2, 1), basis(2, 1))).unit()
#----------------------------------- Initializing first population ensemble
#ansatzket1 = tensor(basis(2),basis(2))
#ansatzket1 = hadamard_transform(N=2) * ansatzket1
#ansatzket2 = tensor(basis(2),basis(2,1))
#ansatzket2 = hadamard_transform(N=2) * ansatzket2
#ansatzket3 = tensor(basis(2,1),basis(2))
#ansatzket3 = hadamard_transform(N=2) * ansatzket3
#ansatzket4 = tensor(basis(2,1),basis(2,1))
#ansatzket4 = hadamard_transform(N=2) * ansatzket4

#ansatzp = 0.25
#ansatzkets = np.array([ansatzket1, ansatzket2, ansatzket3, ansatzket4])
#ansatzps = np.array([ansatzp, ansatzp, ansatzp, ansatzp])
#bin_ansatzps = []
#for p in ansatzps:
#    p = np.binary_repr(int(p*(2**DNA_SIZE-1)), width=DNA_SIZE)
#    p = np.array([int(p[i]) for i in range(len(p))])
#    bin_ansatzps.append(p)
#bin_ansatzps = np.array(bin.ansatzps)


pop_XY = []
pop_p = []
for i in range(POP_SIZE):
    pop_XY.append(np.random.randint(2, size=([INPUT_STATES, 2, 4, DNA_SIZE])))
    pop_p.append(np.random.randint(2, size=([INPUT_STATES, DNA_SIZE])))
pop = [np.array(pop_XY), np.array(pop_p)]
#pop[0][0] =  Ansatz[0]
#pop[1][0] =  Ansatz[1]

#-----------------------------------For plotting
plt.ion()
fig, ax = plt.subplots()
gen, holevo = [],[]
ax.scatter(gen,holevo)
plt.xlim(0,N_GENERATIONS)
#plt.ylim(0.0,1.5)
XN = calc_XN(p) # this is X(N). You need to plot 2X(N)             #amp damp channel
plt.plot([i for i in range(N_GENERATIONS)], [2*XN for j in range(N_GENERATIONS)])   #amp damp channel
#XN = 1-p    #qubit erasure channel
#plt.plot([i for i in range(N_GENERATIONS)], [2*XN for j in range(N_GENERATIONS)])   #qubit erasure channel
#XN = 1 + (1-p+0.5*p) * log2(1-p+0.5*p) + 0.5*p*log2(0.5*p)   #depolarizing channel
#plt.plot([i for i in range(N_GENERATIONS)], [2*XN for j in range(N_GENERATIONS)])   #depolarizing channel
plt.draw()
plt.xlabel("Generations")
plt.ylabel("X(N tensor N) estimate")
#-----------------------------

for _ in range(N_GENERATIONS):
    #------------------------
    if len(holevo)>1 and holevo[-1] > 0.6: MUTATION_RATE = 0.005
    if len(holevo)>1 and holevo[-1] > 0.65: MUTATION_RATE = 0.003
    if len(holevo)>1 and holevo[-1] > 0.7: MUTATION_RATE = 0.001
    if _ > 4000: MUTATION_RATE = 0.002
    #------------------------
    
    
    translated = translateDNA(pop)
    F_values = Func(translated[0], translated[1])    # compute function value by extracting DNA
    holevo.append(np.max(F_values))
    print("probabilities ", translated[1][np.argmax(F_values)]) 
    print("states ", translated[0][np.argmax(F_values)]) 
    print("concurrences ", [concurrence(translated[0][np.argmax(F_values)][i]) for i in range(len(translated[0][np.argmax(F_values)]))])
    clear_output(wait=True)
    #-------plot
    ax.scatter(_, holevo[-1], c='red')
    plt.pause(0.05)
    #-------

    # GA part (evolution)
    fitness = get_fitness(F_values) #FIXED
    pop = select(pop, fitness)
    pop_copy = pop.copy()
    pop_copy_2 = pop.copy()   #
    count = 0
    for parent_XY,parent_p in zip(pop[0], pop[1]):
        #------mutation policy
        if len(holevo) > 1 and holevo[-1]==holevo[-2]:
            count = count + 1
        else:
            count = 0
        if count == MUTATION_DISTANCE + 1:
            count = 0
        #-----------
        child_XY,child_p = crossover_and_mutate(parent_XY, parent_p, pop_copy, count == MUTATION_DISTANCE) 
        pop_copy_2[0] = np.concatenate((pop_copy_2[0],child_XY[np.newaxis, ...]),axis=0)
        pop_copy_2[1] = np.concatenate((pop_copy_2[1],child_p[np.newaxis, ...]),axis=0)
        #parent_XY[:] = child_XY       # parent is replaced by its child
        #parent_p[:] = child_p        # parent is replaced by its child
    pop[0] = pop_copy_2[0]
    pop[1] = pop_copy_2[1]

plt.ioff()

KeyboardInterrupt: 

In [37]:
ket1 = tensor(basis(2),basis(2))
ket1 = hadamard_transform(N=2) * ket1
ket2 = tensor(basis(2),basis(2,1))
ket2 = hadamard_transform(N=2) * ket2
ket3 = tensor(basis(2,1),basis(2))
ket3 = hadamard_transform(N=2) * ket3
ket4 = tensor(basis(2,1),basis(2,1))
ket4 = hadamard_transform(N=2) * ket4

p = 0.25
kets = np.array([ket1, ket2, ket3, ket4])
ps = np.array([p, p, p, p])
XNN = entropy_vn(sum(px * A_p2(ket2dm(ketx)) for px,ketx in zip(ps,kets)), base = 2) - sum(px * entropy_vn(A_p2(ket2dm(ketx)), base = 2) for px,ketx in zip(ps,kets))
print(XNN)

0.75675836931


In [ ]:
p = 0.3                #probability p in the amp. damp. channel
DNA_SIZE = 15         # DNA length   # size of each number in the lists X and Y
POP_SIZE = 10          # population size
CROSS_RATE = 0.001        # mating probability (DNA crossover)
MUTATION_RATE = 0.002    # mutation probability
N_GENERATIONS = 5000

#---------------Moved from functions for optimization
K1 = basis(2,0) * basis(2,0).dag() + sqrt(1-p) * basis(2,1) * basis(2,1).dag()
K2 = sqrt(p) * basis(2,0) * basis(2,1).dag()
KA = tensor(K1,K1)
KA_dag = tensor(K1,K1).dag()
KB = tensor(K1,K2)
KB_dag = tensor(K1,K2).dag()
KC = tensor(K2,K1)
KC_dag = tensor(K2,K1).dag()
KD = tensor(K2,K2)
KD_dag = tensor(K2,K2).dag()

#rho_init = tensor(basis(2,0), basis(2,0)) * tensor(basis(2,0), basis(2,0)).dag()

bs = (tensor(basis(2), basis(2))+tensor(basis(2, 1), basis(2, 1))).unit()
#----------------------------------- Initializing first population ensemble
pop_XY = []
pop_p = []
for i in range(POP_SIZE):
    pop_XY.append(np.random.randint(2, size=([INPUT_STATES, 2, 4, DNA_SIZE])))
    pop_p.append(np.random.randint(2, size=([INPUT_STATES, DNA_SIZE])))
pop = [np.array(pop_XY), np.array(pop_p)]
#pop[0][0] =  Ansatz[0]
#pop[1][0] =  Ansatz[1]

#-----------------------------------For plotting
plt.ion()
fig, ax = plt.subplots()
gen, holevo = [],[]
ax.scatter(gen,holevo)
plt.xlim(0,N_GENERATIONS)
#plt.ylim(0.0,1.5)
XN = calc_XN(p) # this is X(N). You need to plot 2X(N)             #amp damp channel
plt.plot([i for i in range(N_GENERATIONS)], [2*XN for j in range(N_GENERATIONS)])   #amp damp channel
#XN = 1-p    #qubit erasure channel
#plt.plot([i for i in range(N_GENERATIONS)], [2*XN for j in range(N_GENERATIONS)])   #qubit erasure channel
#XN = 1 + (1-p+0.5*p) * log2(1-p+0.5*p) + 0.5*p*log2(0.5*p)   #depolarizing channel
#plt.plot([i for i in range(N_GENERATIONS)], [2*XN for j in range(N_GENERATIONS)])   #depolarizing channel
plt.draw()
plt.xlabel("Generations")
plt.ylabel("X(N tensor N) estimate")
#-----------------------------

for _ in range(N_GENERATIONS):
    translated = translateDNA(pop)
    F_values = Func(translated[0], translated[1])    # compute function value by extracting DNA
    holevo.append(np.max(F_values))
    print("probabilities ", translated[1][np.argmax(F_values)]) 
    print("states ", translated[0][np.argmax(F_values)]) 
    print("concurrences ", [concurrence(translated[0][np.argmax(F_values)][i]) for i in range(len(translated[0][np.argmax(F_values)]))])
    clear_output(wait=True)
    #-------plot
    ax.scatter(_, holevo[-1], c='red')
    plt.pause(0.05)
    #-------

    # GA part (evolution)
    fitness = get_fitness(F_values) #FIXED
    pop = select(pop, fitness)
    pop_copy = pop.copy()
    pop_copy_2 = pop.copy()   #
    count = 0
    for parent_XY,parent_p in zip(pop[0], pop[1]):
        #------mutation policy
        if len(holevo) > 1 and holevo[-1]==holevo[-2]:
            count = count + 1
        else:
            count = 0
        if count == 2:
            count = 0
        #-----------
        child_XY,child_p = crossover_and_mutate(parent_XY, parent_p, pop_copy, count > 0) 
        pop_copy_2[0] = np.concatenate((pop_copy_2[0],child_XY[np.newaxis, ...]),axis=0)
        pop_copy_2[1] = np.concatenate((pop_copy_2[1],child_p[np.newaxis, ...]),axis=0)
        #parent_XY[:] = child_XY       # parent is replaced by its child
        #parent_p[:] = child_p        # parent is replaced by its child
    pop[0] = pop_copy_2[0]
    pop[1] = pop_copy_2[1]

plt.ioff()

In [17]:
p = 0.3                #probability p in the amp. damp. channel
DNA_SIZE = 20         # DNA length   # size of each number in the lists X and Y
POP_SIZE = 10          # population size
CROSS_RATE = 0.001        # mating probability (DNA crossover)
MUTATION_RATE = 0.002    # mutation probability
N_GENERATIONS = 5000

#---------------Moved from functions for optimization
K1 = basis(2,0) * basis(2,0).dag() + sqrt(1-p) * basis(2,1) * basis(2,1).dag()
K2 = sqrt(p) * basis(2,0) * basis(2,1).dag()
KA = tensor(K1,K1)
KA_dag = tensor(K1,K1).dag()
KB = tensor(K1,K2)
KB_dag = tensor(K1,K2).dag()
KC = tensor(K2,K1)
KC_dag = tensor(K2,K1).dag()
KD = tensor(K2,K2)
KD_dag = tensor(K2,K2).dag()

#rho_init = tensor(basis(2,0), basis(2,0)) * tensor(basis(2,0), basis(2,0)).dag()

bs = (tensor(basis(2), basis(2))+tensor(basis(2, 1), basis(2, 1))).unit()
#----------------------------------- Initializing first population ensemble
pop_XY = []
pop_p = []
for i in range(POP_SIZE):
    pop_XY.append(np.random.randint(2, size=([INPUT_STATES, 2, 4, DNA_SIZE])))
    pop_p.append(np.random.randint(2, size=([INPUT_STATES, DNA_SIZE])))
pop = [np.array(pop_XY), np.array(pop_p)]
#pop[0][0] =  Ansatz[0]
#pop[1][0] =  Ansatz[1]

#-----------------------------------For plotting
plt.ion()
fig, ax = plt.subplots()
gen, holevo = [],[]
ax.scatter(gen,holevo)
plt.xlim(0,N_GENERATIONS)
#plt.ylim(0.0,1.5)
XN = calc_XN(p) # this is X(N). You need to plot 2X(N)             #amp damp channel
plt.plot([i for i in range(N_GENERATIONS)], [2*XN for j in range(N_GENERATIONS)])   #amp damp channel
#XN = 1-p    #qubit erasure channel
#plt.plot([i for i in range(N_GENERATIONS)], [2*XN for j in range(N_GENERATIONS)])   #qubit erasure channel
#XN = 1 + (1-p+0.5*p) * log2(1-p+0.5*p) + 0.5*p*log2(0.5*p)   #depolarizing channel
#plt.plot([i for i in range(N_GENERATIONS)], [2*XN for j in range(N_GENERATIONS)])   #depolarizing channel
plt.draw()
plt.xlabel("Generations")
plt.ylabel("X(N tensor N) estimate")
#-----------------------------

for _ in range(N_GENERATIONS):
    translated = translateDNA(pop)
    F_values = Func(translated[0], translated[1])    # compute function value by extracting DNA
    holevo.append(np.max(F_values))
    print("probabilities ", translated[1][np.argmax(F_values)]) 
    print("states ", translated[0][np.argmax(F_values)]) 
    print("concurrences ", [concurrence(translated[0][np.argmax(F_values)][i]) for i in range(len(translated[0][np.argmax(F_values)]))])
    clear_output(wait=True)
    #-------plot
    ax.scatter(_, holevo[-1], c='red')
    plt.pause(0.05)
    #-------

    # GA part (evolution)
    fitness = get_fitness(F_values) #FIXED
    pop = select(pop, fitness)
    pop_copy = pop.copy()
    pop_copy_2 = pop.copy()   #
    count = 0
    for parent_XY,parent_p in zip(pop[0], pop[1]):
        #------mutation policy
        if len(holevo) > 1 and holevo[-1]==holevo[-2]:
            count = count + 1
        else:
            count = 0
        if count == 2:
            count = 0
        #-----------
        child_XY,child_p = crossover_and_mutate(parent_XY, parent_p, pop_copy, count > 0) 
        pop_copy_2[0] = np.concatenate((pop_copy_2[0],child_XY[np.newaxis, ...]),axis=0)
        pop_copy_2[1] = np.concatenate((pop_copy_2[1],child_p[np.newaxis, ...]),axis=0)
        #parent_XY[:] = child_XY       # parent is replaced by its child
        #parent_p[:] = child_p        # parent is replaced by its child
    pop[0] = pop_copy_2[0]
    pop[1] = pop_copy_2[1]

plt.ioff()

probabilities  [ 0.24795508  0.24873679  0.25164319  0.25166494]
states  [ Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[-0.04146466+0.64189882j]
 [-0.16212390+0.29765476j]
 [-0.49238887-0.32837162j]
 [-0.12613086-0.32431521j]]
 Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[ -1.85789836e-05-0.42947015j]
 [ -5.73431587e-01+0.39222803j]
 [  0.00000000e+00+0.28523275j]
 [ -1.27757835e-03-0.5015271j ]]
 Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[-0.48546306 +3.19870577e-01j]
 [-0.22828384 -4.66883205e-01j]
 [-0.01548248 +4.59051714e-01j]
 [-0.42537841 +1.34058862e-04j]]
 Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[-0.09662437 -5.24949967e-01j]
 [-0.39661467 -5.00701495e-07j]
 [-0.52584072 -2.45758814e-01j]
 [-0.25378656 +3.95568701e-01j]]]
concurrences  [0.088328839441352766, 0.38807459809250328, 0.080427700618477968, 0.047496649444804417]


In [ ]:
#Ansatz = [pop[0][1] , pop[1][1]]